In [1]:

# This cell is to load user and password of my database from an .txt file, which will not be pushed to the repository
# Assign your user name and password to lldb_user and lldb_password variables
with open('db_credentials.txt','r') as file:
    lines = file.readlines()
lldb_user = lines[0]
lldb_password = lines[1]
    

In [2]:
import mysql.connector as connector
from mysql.connector import Error

In [3]:
try:
    connection = connector.connect(user = lldb_user, password = lldb_password, db = 'littlelemondb')
except Error as e:
    print('An error has occurred with an error code: {} \n and message: {}'.format(e.errno,e.msg))

In [4]:
cursor = connection.cursor()
cursor.execute('SHOW tables')

In [5]:
results = cursor.fetchall()
for table in results:
    print(table)

('bookings',)
('customers',)
('deliveries',)
('items',)
('menuitems',)
('menus',)
('orders',)
('ordersview',)
('staff',)


In [6]:
cursor.execute("""
SELECT CONCAT(bc.FirstName, ' ', bc.LastName) AS 'Customer Full Name', bc.Email, bc.Phone, o.TotalCost
FROM Orders AS o 
LEFT JOIN 
	(SELECT b.BookingID, c.CustomerID, c.FirstName, c.LastName, c.Email, c.Phone
    FROM Bookings AS b
    LEFT JOIN Customers AS c 
    ON b.CustomerID = c.CustomerID) AS bc
ON o.BookingID = bc.BookingID
WHERE o.TotalCost > 60;
""")

print(cursor.column_names)

for record in cursor.fetchall():
    print (record)

('Customer Full Name', 'Email', 'Phone', 'TotalCost')
('Vanessa McCarthy', 'vanessa.m@globalsuperstore.com', 361255597, Decimal('92'))
('Joakim Iversen', 'j.iversen@yahoo.com', 352253781, Decimal('78'))
('Anna Iversen', 'anna.iversen@gmail.com', 351259642, Decimal('280'))
('Anna Iversen', 'anna.iversen@gmail.com', 351259642, Decimal('138'))
('Vanessa McCarthy', 'vanessa.m@globalsuperstore.com', 361255597, Decimal('120'))
('Diana Pinto', 'diana.pinto@finance.org.us', 751100242, Decimal('184'))


In [7]:
def CallStoredProcedure(procedureCallQuery, parameters_tuple = None):
    try:
        cursor = connection.cursor()
        if parameters_tuple == None:
            cursor.execute(procedureCallQuery)
        else:
            cursor.execute(procedureCallQuery, parameters_tuple)
        while True:
            print (cursor.column_names)
            for record in cursor.fetchall():
                print(record)
            if not cursor.nextset():
                break
    except Error as e:
        print('An error has occurred with an error code: {} \n and message: {}'.format(e.errno,e.msg))  
    finally:
        cursor.close()

In [8]:
CallStoredProcedure("CALL GetMaxQuantity();")

('Max Quantity in Order',)
(5,)
()


In [9]:
CallStoredProcedure("CALL ManageBooking(%s,%s);",("2025-02-28",6))

('Booking Status',)
('Table 6 has been booked for 2025-02-28.',)
()


In [10]:
CallStoredProcedure("CALL ManageBooking(%s,%s);",("2025-02-28",6))

('Booking Status',)
('Table 6 is already booked on 2025-02-28 - booking cancelled.',)
()


In [11]:
CallStoredProcedure("CALL ManageBooking(%s,%s);",("2022-10-10",5))

('Booking Status',)
('Table 5 is already booked on 2022-10-10 - booking cancelled.',)
()


In [12]:
CallStoredProcedure("CALL AddBooking(%s,%s,%s,%s);",(99, 99, 99,"2022-12-10" ))

('Booking Status',)
('Table 99 has been booked for 2022-12-10. New booking added.',)
()


In [13]:
CallStoredProcedure("CALL AddBooking(%s,%s,%s,%s);",(99, 99, 99,"2022-12-10" ))

('Message',)
('The BookingID - 99 has been taken. Select another one.',)
()


In [14]:
CallStoredProcedure("CALL UpdateBooking(%s,%s);",(99, "2022-1-10"))

('Confirmation',)
('Booking 99 updated.',)
()


In [15]:
CallStoredProcedure("CALL CancelBooking(%s);",(99,))

('Confirmation',)
('Booking 99 cancelled.',)
()


In [16]:
# Close the connection
if connection.is_connected():
    cursor.close()
    connection.close()
    print("Connection has been closed")
else:
    print("Connection was closed")

Connection has been closed
